## Regression Intro

In [55]:
import pandas as pd
import yfinance as yf
import math

In [56]:
df = yf.download('GOOGL', start='2010-01-01', end='2024-12-31')
df.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GOOGL,GOOGL,GOOGL,GOOGL,GOOGL
Date,,,,,
2010-01-04,15.609804,15.678545,15.547291,15.614785,78169752
2010-01-05,15.541064,15.636953,15.480045,15.620515,120067812
2010-01-06,15.149293,15.587637,15.101972,15.587637,158988852
2010-01-07,14.796625,15.192631,14.760512,15.177686,256315428
2010-01-08,14.993881,15.024515,14.672344,14.744323,188783028


In [57]:
# high minus low percent
df["HL_PCT"] = (df['High'] - df['Low']) / df['Low'] * 100

# daily percent change
df["PCT_change"] = (df['Close'] - df['Open']) / df['Open'] * 100

df.head()

Price,Close,High,Low,Open,Volume,HL_PCT,PCT_change
Ticker,GOOGL,GOOGL,GOOGL,GOOGL,GOOGL,,
Date,,,,,,,
2010-01-04,15.609804,15.678545,15.547291,15.614785,78169752,0.844225,-0.031900
2010-01-05,15.541064,15.636953,15.480045,15.620515,120067812,1.013611,-0.508628
2010-01-06,15.149293,15.587637,15.101972,15.587637,158988852,3.215908,-2.812129
2010-01-07,14.796625,15.192631,14.760512,15.177686,256315428,2.927528,-2.510668
2010-01-08,14.993881,15.024515,14.672344,14.744323,188783028,2.400233,1.692573


In [58]:
df = df[['Close','HL_PCT',"PCT_change",'Volume']]
df.head()

Price,Close,HL_PCT,PCT_change,Volume
Ticker,GOOGL,,,GOOGL
Date,,,,
2010-01-04,15.609804,0.844225,-0.031900,78169752
2010-01-05,15.541064,1.013611,-0.508628,120067812
2010-01-06,15.149293,3.215908,-2.812129,158988852
2010-01-07,14.796625,2.927528,-2.510668,256315428
2010-01-08,14.993881,2.400233,1.692573,188783028


In [59]:
forecast_col = 'Close'
df.fillna(-99999, inplace=True) # it helps to keep data as an outlier instead of get rid of the data points

In [60]:
forecast_out = int(math.ceil(0.01*len(df)))
forecast_out

38

In [61]:
df['label'] = df['Close'].shift(-forecast_out)
df.head()

Price,Close,HL_PCT,PCT_change,Volume,label
Ticker,GOOGL,,,GOOGL,
Date,,,,,
2010-01-04,15.609804,0.844225,-0.031900,78169752,13.267151
2010-01-05,15.541064,1.013611,-0.508628,120067812,13.475614
2010-01-06,15.149293,3.215908,-2.812129,158988852,13.581715
2010-01-07,14.796625,2.927528,-2.510668,256315428,13.812591
2010-01-08,14.993881,2.400233,1.692573,188783028,14.052185
